In [6]:
news_data = []
with open("mockstocknews.txt", "r", encoding="utf-8") as file:
    news_data = file.readlines()
    print(f"loaded {len(news_data)}")

loaded 46


In [9]:
import ollama
embedding_model = "hf.co/CompendiumLabs/bge-base-en-v1.5-gguf"
language_model = "hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF"

vector_db = []

In [12]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=embedding_model, input=chunk)["embeddings"][0]
    vector_db.append((chunk, embedding))

In [13]:
for i, chunk in enumerate(news_data):
    add_chunk_to_database(chunk)
    print(f"added chunk {i+1}/{len(news_data)} to the database")

added chunk 1/46 to the database
added chunk 2/46 to the database
added chunk 3/46 to the database
added chunk 4/46 to the database
added chunk 5/46 to the database
added chunk 6/46 to the database
added chunk 7/46 to the database
added chunk 8/46 to the database
added chunk 9/46 to the database
added chunk 10/46 to the database
added chunk 11/46 to the database
added chunk 12/46 to the database
added chunk 13/46 to the database
added chunk 14/46 to the database
added chunk 15/46 to the database
added chunk 16/46 to the database
added chunk 17/46 to the database
added chunk 18/46 to the database
added chunk 19/46 to the database
added chunk 20/46 to the database
added chunk 21/46 to the database
added chunk 22/46 to the database
added chunk 23/46 to the database
added chunk 24/46 to the database
added chunk 25/46 to the database
added chunk 26/46 to the database
added chunk 27/46 to the database
added chunk 28/46 to the database
added chunk 29/46 to the database
added chunk 30/46 to th

In [14]:
# use cosine similiarity to find close chunks 
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [16]:
def retrieve(query, top_n=3):
    query_embedding = ollama.embed(model=embedding_model, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in vector_db:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    # sort by similarity in descending order, because higher similarity means more relevant chunks
    similarities.sort(key=lambda x: x[1], reverse=True)
    # finally, return the top N most relevant chunks
    return similarities[:top_n]

In [ ]:
input_query = input("Ask me anything about stocks! ")
retrieved_knowledge = retrieve(input_query)

print("retrieved knowledge: ")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")

context_text = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f"""
You are a stock market analysis assistant with access to real-time news via RAG.

Rules:
1. Use ONLY the retrieved news articles as your information source.
2. Always rely on the LATEST news provided in the feed. Do not speculate or fabricate numbers, prices, or events.
3. Provide DETAILED responses, not short summaries. Expand on:
   - The context behind the news
   - Potential short-term and long-term impacts on individual stocks, sectors, or the overall market
   - Sentiment trends (positive, negative, mixed)
   - Broader macroeconomic or geopolitical implications if relevant
4. When multiple articles are retrieved:
   - Identify common themes across them
   - Compare differences in tone, sentiment, or perspective
   - Highlight potential risks and opportunities
5. Clearly separate FACTS from INTERPRETATION:
   - Use “According to the article…” for factual reporting
   - Use “This could suggest…” or “This may impact…” for analysis
6. If no useful financial information is present in the retrieved news, explicitly state that, instead of guessing or fabricating.
7. Write in a professional, analytical style similar to financial research notes, ensuring your responses are thorough and actionable.

Response Format:
Start with a section called "Retrieved Articles," listing the exact articles retrieved with RAG (title, source, and short snippet).
Then continue with long-form analysis written in paragraphs. 
Structure the response naturally with flowing sections:
- Do not use Markdown, bold, or bullet points.
- Begin with a summary of the main news points.
- Then discuss potential market impacts in detail, covering both short-term and long-term implications.
- Provide a discussion of sentiment, noting whether the overall tone is positive, negative, or mixed.
- Conclude with a paragraph that integrates risks and opportunities into a forward-looking outlook, rather than labeling them as bullet points.

Your goal is to generate longer, well-structured, and insightful analysis that helps users understand how the most recent news may influence stocks or sectors, while staying strictly grounded in the retrieved articles.
Use only the following pieces of context to answer the question. Don't make up any new information:
{context_text}
"""

retrieved knowledge: 
 - (similarity: 0.66) Mitsubishi UFJ Asset Management Co. Ltd. raised its position in Apple Inc. (NASDAQ:AAPL – Free Report) by 11.3% during the first quarter, Holdings Channel reports.

 - (similarity: 0.65) Wittenberg Investment Management Inc.’s holdings in Apple were worth $24,045,000 at the end of the most recent quarter.

 - (similarity: 0.64) Mitsubishi UFJ Asset Management Co. Ltd.’s holdings in Apple were worth $6,096,892,000 as of its most recent SEC filing.



In [49]:
stream = ollama.chat(
    model=language_model,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
Retrieved Articles:

* "Mitsubishi UFJ Asset Management Co. Ltd. raised its position in Apple Inc. (NASDAQ:AAPL – Free Report) by 11.3% during the first quarter, Holdings Channel reports."
* "Wittenberg Investment Management Inc.’s holdings in Apple were worth $24,045,000 at the end of the most recent quarter."
* "Mitsubishi UFJ Asset Management Co. Ltd.’s holdings in Apple were worth $6,096,892,000 as of its most recent SEC filing."

Context:
Apple Inc. (NASDAQ:AAPL) has been a top performer in recent quarters, driven by strong demand for iPhones and MacBooks, as well as growth in the company's services segment.

Short-term Impacts:

* The 11.3% increase in Mitsubishi UFJ Asset Management Co. Ltd.'s position in Apple suggests that investors remain bullish on the stock.
* Wittenberg Investment Management Inc.'s holdings of $24,045,000 indicate a significant allocation to Apple, which may drive future buying activity.

Long-term Impacts:

* The relatively modest gains 